# Making SAFE More Secure

As discussed in the Section ..., it may be desirable to make the SAFE Protocol more secure against privacy-invasing information flows that could result from the aggregator's knowledge of the interval `[-D, D]`. 

This notebook illustrates a way to make privacy-invading bound inferences on the private values of users via randomizing the `[-D, D]` on each protocl run. 

The intent is not to thorough or completely rigorous; each step in the suggested procedure will throw up a host of issues that, to fully resolve, would take us beyond the scope of the project. The aim is to sketch a **proof of concept** and nudge in the direction of how to develop this approach fully to maximal security.

[add specific section]:# (add section in the above)

Comments will be made on the limits this randomization introduces.

## Randomization Algorithm

As described in Section ..., one particular information flow that a semi-honest aggregator could exploit to infer bounds on private values rests on their knowledge of the doubles interval `[-D, D]` they know users draw random shares from.

This suggests a way to make SAFE more secure: randomize `[-D, D]` in each protocol run such that the aggregator would not be able to leverage knowledge of `[-D, D]` to make those inferences.

One way to do this would be to:
* Assume all users begin with some shared secret seed
* Using this shared seed, they generate a bit sequence using some given Pseudorandom Generator
* Map this bit sequence to a real value using some deterministic algorithm
* Use this real value as `D` in the `[-D, D]` interval.

The effcet of this will be that each protocl run executes with a `D` value the aggregator is not able to know, thus preventing inferences on the basis of `[-D, D]`.

Let us consider this procedure in greater detail. 

## Shared Secret Seed

Pseudorandom Generators (PRNG) are algorithms that generate a sequence of numbers that approximate a sequence of truly random numbers.{cite}`wiki:Pseudorandom_number_generator`. They are 'pseudorandom' because the sequence they generate is entirely determined by the initial seed value that is passed in. This seed value initialises the *internal state* of the PRNG. PRNGs use their internal state in their algorithms to generate the pseudorandom output. After each generation of an output, their internal state gets updated.  

The important point is that if users use the same seed for the same PRNG, each time they invoke the PRNG, they will end up with the same output. That means if users can all seed the same PRNG with the same seed, they can derive the same random output. By mapping this output to a real value, they will effectively have a random `D` value to constitute `[-D, D]` interval from which they draw their random shares.

For this to achieve the intended effect of making the SAFE protocol more secure, a few conditions must be met:
* users must have a shared secret seed that the aggregator does not learn of
* users must use the same PRNG
* there must be an effective way to map the output of a PRNG to a real value suitable for use as the `D` value

### Distributing A Shared Secret Seed

Unfortunately, considering this first step takes us to a host of issues that we must consider. These issues relate to the general problem of secret establishment among many (i.e., more than two parties).

In this section, we will consider a variety of protocols for establishing the shared secret seed. 

Ultimately, no protocol comes without limitations and additional complexities. However, these are important considerations when deciding whether or not the greater security afforded by this proposed algorithm is worth the costs.

The following subsections will discuss various protocols. To skip ahead to a tentative decision and proceed with see the rest of the procedure, click [here](#shared-pseudorandom-generator--getting-a-real)

#### Diffe-Helman Key Exchange

Let us first consider how users might establish a secret seed by via communicating with one another.

To do this, we can first consider the Diffe-Helman Key Exchange. We will see that this protocol in its simplest form is not a viable solution,however, it is worth explicating and considering it first as it will raise certain issues and themes that will be picked up in subsequent protocols.

The Diffe-Helman Key Exchange is a means for two parties to establish some secure key in an insecure channel. This section's basic description of the protocol relies on {cite}`wiki:Diffie–Hellman_key_exchange`

##### Two Users 

For instance, suppose we have just two users in engaged in the SAFE protocol run, Alice and Bob. 

Both users publically agree on a $p$ and $g$ value, where $p$ is some prime and $g$ is the primitive root modulo $p$ or *group generator* for the multiplicative group of integers modulo $p$. 

Both Alice and Bob each choose some secret integer. Let $a$ denote Alice's secret and $b$ denote Bob's secret.

Alice computes $A = g^a  \space mod  \space p$ and sends $A$ to Bob, while Bob computes $B = g^b  \space mod  \space p$ and sends $B$ to Alice,

Then, Alice computes $B^a  \space mod \space p$ and Bob computes $A^b  \space mod  \space p$.

Becuase

$$
(g^a mod p)^b mod  \space p = (g^b mod p)^a mod  \space p 
$$

, both Alice and Bob would have arrived at the same value, $S$.

Even if the aggregator knows $g$, $p$, $A$ and $B$, it is very hard for them to derive the $S$ that Alice and Bob now share, so long as $a$ and $b$ remain hidden.

How hard is 'very hard'? The hardness of deriving $S$ consists in solving the *discrete logarithmic problem*, a problem deemed very hard for even modern supercomputers.

##### $N$ (where $N > 2$) Users

Of course, our use case is not likely to involve just two users, but many, many more. Is such a protocol scalable?

Assuming we try to simply scale the bare version of the protocol, we end up with a fairly untenable growth in complexity. More specifically, with $N$ users, each user will do $N$ exponentiations, meaning that there'll be a total of $N^2$ exponentiations for $N$ users.{cite}`se:generalize-diffe-helman`

An exponential growth in operations is probably undesirable even if the use-case is constrained to a relatively small scale. So, let us consider alternative protocols.

##### Using a Key Distribution Protocol

An alternative to having each user communicate with every other user is to designate some trusted server or chairman who distributes the secret seed to each user. However, this, among other things, merely introduces another third party and for whom undesirable information flows (assuming a semi-honest model) could result from. So, this option is a non-starter. 

##### Using A Key Agreement Protocol

One alternative protocol comes from {cite}`10.1093/comjnl/bxh111`. Tseng presents a multi-party key agreement protocol which allows $N$ users to collaboratively establish a common key without a designated chairman. 

The general idea behind the protocol is this: each user begins with two broadcast round. In each broadcast round, they calculate some random variable labelled with their user-id and send it to all other participants. The results of the first broadcast round are used to calculate the results distributed during the second broadcast round.

After receiving the second broadcast round variates from all other users, each user can compute two checks to see if there is any malicious participant in the protocol run. This check is user-specific, so pinpointing who the malicious participant is can be done. However, to save computational costs, users could instead bypass these initial checks and compute the common key $K$ according to some formula. This would be what happens if the two checks raise no issues anyway.

Then, they check if they can decrypt encrypted conference messages. If not, then there is a malicious particpant who has disrupted the conference and they must run the fault deduction procedure, and oust the malicious participant. If userse can decrypt conference messages using the constructed key $K$, that means there are no malicious participants in this conference round.

Tseng proves that the total computational cost of this protocol for each participant is $O(N)$ for $N$ participants, resistant to malicious participants. Moreover, number of messaging rounds and message size stay constant as $N$ grows. (ibid., 477)  

##### Usable in SAFE?

As promising as this protocol is, there are some things to consider with applying it in the SAFE protocol, and particularly for our use-case.

It is stipulated in Tseng's protocol that each user knows that set $U$ where $U$ is the set of initial participants wanting to generate the common conference key. (ibid., 471) However, this raises another general issue to reckon with in key distributions: authentication. Even assuming that we set up the infrastructure and resources required for communication, how do users know that those participating in the conference are in fact users? How do they know the aggregator is not included in the conference?

Tsung states that this protocol is a non-authenticated multi-party key agreement, which, "by its very nature cannot provide provide participant and message authentication, so it must rely on the authenticated network channel, or use another scheme to provide authentication." (ibid., 471) So, one way forwad is to assume that the users are part of an authenticated broadcast channel, and thus that it is secure against an impersonating aggregator (ibid.). Or, as suggested by Teng (ibid), one could apply the compiler presented in {cite}`10.1007/978-3-540-45146-4_7`. This compiler "transforms any group key-exchange protocol secure against a passive eavesdropper to an authenticated protocol which is secure against an active adversary who controls all communication in the network." (ibid., 110)

So, the problem of authentication is not insurmountable. But, either way, it's clear that trying to run this introduces some additional complexities, which will need to be weighted against other constraints. 

Another key consideration for its use in SAFE is that there aren't any conference messages for users to decrypt anyway. Instead, all that is needed is for users to generate some common seed.

Assuming a semi-honest model (which is what SAFE assumes), this is tenable because all users will conform to the protocol and compute the secret key accordingly. So, we could use the operations proposed in the paper to generate the requisite seed.

[even though Dr Huth has responded and given a suggestion, I don't think I'm going to take it up]:# (doesn't seem like a non-communication style is feasible)

##### Further Work

A fuller and more rigorious analysis of the various issues would be in order if one really were pressed to ensure both the scalability an security of this procedure run over the SAFE protocol. That is beyond the scope of this project, so I defer it for future work.

We have from {cite}`10.1093/comjnl/bxh111` a viable protocol for our purposes, so I will work simply assume that the $N$ users have established the secret seed by the means of the key agreement protocol, and consider the next step.


## Shared Pseudorandom Generator & Getting A Real

The distribution of the shared PRNG is far simpler than the distribution of the secret seed, since it can be public knowledge. It is acceptable that the aggregator knows what is the pseudorandom generator in use, so long as the seed parameter remains secret.

### Picking a Pseudo-Random Generator

Broadly speaking, there are two variants of PRNGs: those that are designed to be cryptographically-secure and those that are not.

Since we are just engaing in a proof-of-concept, I will illustrate the protocol with a blatantly non-secure PRNG: the one from Python's `random` module.{cite}`python-docs-random`. Python also provides a PRNG that is meant for security purposes in their `secrets` module.{cite}`python-docs-secrets`. However, this cannot be used in our context because it is not possible to manually seed the PRNG. 

If we wish to use a CPRNG, we need one that can be manually seeded such that the users can use their shared secret seed to initialise the CPRNG, and thus end up with the same random `D` value for each run of the SAFE protocol.

Further discussion of the availability of a CPRNG that can be used is found in Section {...} of the report.

### Transforming Output of PRNG Into Suitable Float

Assuming that users use the same PRNG (the one in `random`) and a shared seed, they can generate a shared random `D` value for each round of the SAFE protocol. 

Below we adapt a PRNG from the documentation to generate the random floats.

In [47]:
# this code is adapted from the following source:

"""
* Author(s) name : Python Software Foundation
* Date Accessed: 22 August 2022
* Title : 'Recipes' in "random-Generate pseudo-random numbers"
* Code Version: as in https://github.com/python/cpython/blob/3.10/Lib/random.py
* Type: source code
* Web address: https://docs.python.org/3/library/random.html

To quote from the documentation, using an instance of the FullRandom class,

" All floats in the interval are possible selections. The mantissa comes from a uniform distribution of integers in the range 2⁵² ≤ mantissa < 2⁵³. The exponent comes from a geometric distribution where exponents smaller than -53 occur half as often as the next larger exponent"

We adapt the code to generate randoms where the mantissa comes from the same uniform distribution as stated in the quote, but modify the geoemtric distribution the exponents come from to be significantly less so we raise the chances of a large D value.

"""

from random import Random
from math import ldexp

# as given in the documentation
manitssa_bit_size = 52 # value in documentation example: 52
exponent_size = -3 # value in documentation example: -53. With 

class FullRandom(Random):

    def random(self):
        mantissa = 0x10_0000_0000_0000 | self.getrandbits(manitssa_bit_size)
        exponent = exponent_size
        x = 0
        while not x:
            x = self.getrandbits(32)
            exponent += x.bit_length() - 32
        return ldexp(mantissa, exponent)
        # https://www.geeksforgeeks.org/python-ldexp-function/#:~:text=ldexp%20%28%29%20function%20is%20one%20of%20the%20Standard,Python%20frexp%20%28%29%20function.%20Syntax%3A%20math.ldexp%20%28x%2C%20i%29

# maybe I can use this to randomize the manitssa and exponent fields.

fr = FullRandom()
shared_random_seed = 1153
fr.seed(shared_random_seed)

# for formatting:
places = 4
for i in range(100):
    print(f'Round {i} D value: {fr.random():.{places}f}')



Round 0 D value: 798073140527838.3750
Round 1 D value: 527028624112267.8125
Round 2 D value: 976355506417123.5000
Round 3 D value: 1025814227865377.1250
Round 4 D value: 835175841480290.7500
Round 5 D value: 1011673140354178.2500
Round 6 D value: 107949891321746.4062
Round 7 D value: 190354830287074.6250
Round 8 D value: 887157341839180.1250
Round 9 D value: 744224403171983.7500
Round 10 D value: 318267061583398.6250
Round 11 D value: 968520634624478.2500
Round 12 D value: 875707855574165.8750
Round 13 D value: 1096325994516583.6250
Round 14 D value: 938532370201910.7500
Round 15 D value: 92361599134833.6094
Round 16 D value: 434840091897406.1875
Round 17 D value: 724863997828325.6250
Round 18 D value: 937969460575424.7500
Round 19 D value: 911648878525915.5000
Round 20 D value: 32173228665853.7773
Round 21 D value: 540760099784526.2500
Round 22 D value: 632157177990430.5000
Round 23 D value: 774685718833875.5000
Round 24 D value: 957047887793597.6250
Round 25 D value: 1017499719090109

We now have a way for users to arrive at the same `D` value to run the protocol with, while the aggregator will not be able to reconstruct this same `D` value so long as the shared seed value remains secret.

## Putting Everything Together

Let's bring everything together and see our procedure generate a few `D` values. We will generate 10.

In [ ]:
# assume users have random_seed

random_seed = 165343 

# generate 10 random outputs

random_outputs = []

# convert to D values



## Test Runs of SAFE With Random `[-D, D]` Interval

Now, let's consider the perspective of the aggregator trying to make bound inferences on a user's private value given 10 runs of the SAFE Protocol with randomized `D` values. 

We will look at the obfuscated feature vector he receives in each round and consider whether information flows of the kind mentioned in Section ..., are still possible.


### Bibliography

[needed to include this bibliography tag to get citations to work: found that from https://github.com/executablebooks/jupyter-book/issues/1662]:# (why this bibliography code block is needed)

```{bibliography}
```